In [1]:
%cd ~/cma/CMA_Fairness_v2

/dss/dsshome1/0C/ra93lal2/cma/CMA_Fairness_v2


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /dss/dsshome1/0C/ra93lal2/cma/CMA_Fairness_v2


## Setup

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

## 01 Data Checks

In [8]:
siab = pd.read_csv("data/raw/siab.csv")

In [9]:
# Compute summary statistics
siab.describe(include = 'all')

,persnr,year,nrEntry,ltue,employed_before,receipt_leh_before,receipt_lhg_before,se_before,ASU_notue_seeking_before,ASU_other_before,...,minijob_tot_dur_byage,ft_tot_dur_byage,befrist_tot_dur_byage,leih_tot_dur_byage,LHG_tot_dur_byage,LEH_tot_dur_byage,almp_tot_dur_byage,almp_aw_tot_dur_byage,se_tot_dur_byage,seeking1_tot_dur_byage
count,6.436900e+05,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,...,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000,643690.000000
mean,9.133846e+05,2012.895383,2.113809,0.151624,0.478210,0.059805,0.373691,0.105886,0.623934,0.150257,...,9.867404,17.794624,3.672047,0.816308,18.417392,14.090453,7.798362,0.953966,2.623641,18.136723
std,5.285426e+05,2.030276,1.487854,0.358657,0.499525,0.237126,0.483783,0.307692,0.484397,0.357324,...,17.086982,27.408078,9.240998,3.747576,24.383040,19.815453,12.396051,2.665374,5.814336,21.168482
min,7.000000e+00,2010.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.559420e+05,2011.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.138889
50%,9.132190e+05,2013.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,1.621622,5.846154,0.000000,0.000000,4.527273,6.594595,2.080000,0.000000,0.000000,10.243243
75%,1.371612e+06,2015.000000,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,12.464286,24.297729,0.738095,0.000000,32.800000,19.800000,10.870968,0.320000,2.279767,27.795802
max,1.827869e+06,2016.000000,15.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,181.264706,664.867925,94.090909,78.909091,139.269231,175.245283,186.750000,78.714286,95.620690,157.473684


In [10]:
# Count number of missing values
siab.isna().sum()

persnr                    0
year                      0
nrEntry                   0
ltue                      0
employed_before           0
                         ..
LEH_tot_dur_byage         0
almp_tot_dur_byage        0
almp_aw_tot_dur_byage     0
se_tot_dur_byage          0
seeking1_tot_dur_byage    0
Length: 164, dtype: int64

In [12]:
grouped = siab.groupby('year')
siab_s = grouped.apply(lambda x: x.sample(n = 15000, random_state = 42))
siab_s = siab_s.reset_index(drop = True) # Ungroup

/tmp/ipykernel_3352743/2218671130.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  siab_s = grouped.apply(lambda x: x.sample(n = 15000, random_state = 42))


In [13]:
siab_s.groupby('year').describe(include = 'all')

persnr                                                            \
        count           mean            std    min        25%       50%   
year                                                                      
2010  15000.0  907830.166933  524912.666601  533.0  457252.00  909817.0   
2011  15000.0  912588.757067  527006.775031  685.0  458636.75  907035.5   
2012  15000.0  908427.682267  527223.584469  142.0  455513.50  904410.0   
2013  15000.0  910627.377467  527436.132336  253.0  454584.75  905396.5   
2014  15000.0  911227.001467  526762.848111  910.0  457839.00  903839.5   
2015  15000.0  917889.995067  526685.665074  917.0  463912.00  915341.0   
2016  15000.0  910381.202533  524203.044681  260.0  455933.50  907327.5   

                             nrEntry            ... se_tot_dur_byage  \
             75%        max    count      mean  ...              75%   
year                                            ...                    
2010  1359947.00  1827770.0  15000.0  1.141000  ...         2.400000   
2011  1368037.75  1827844.0  15000.0  1.577467  ...         2.656473   
2012  1361564.25  1827859.0  15000.0  1.938600  ...         2.315714   
2013  1370434.00  1827869.0  15000.0  2.246600  ...         2.190476   
2014  1366497.00  1827869.0  15000.0  2.525133  ...         2.255963   
2015  1377154.75  1827860.0  15000.0  2.762600  ...         2.138332   
2016  1366129.75  1827860.0  15000.0  2.873267  ...         2.021579   

                seeking1_tot_dur_byage                                       \
            max                  count       mean        std  min       25%   
year                                                                          
2010  43.128205                15000.0  16.308566  18.656543  0.0  1.103448   
2011  64.500000                15000.0  17.485621  19.427911  0.0  1.618831   
2012  60.333333                15000.0  18.296520  20.506406  0.0  1.666667   
2013  89.307692                15000.0  18.687464  21.358468  0.0  1.458548   
2014  74.730769                15000.0  19.377765  22.409588  0.0  1.363636   
2015  93.571429                15000.0  19.091188  22.860249  0.0  0.878603   
2016  65.607143                15000.0  18.140198  22.915966  0.0  0.000000   

                                        
            50%        75%         max  
year                                    
2010   9.638182  24.962063  114.764706  
2011  10.592793  27.009615  113.515152  
2012  11.000000  28.242500  116.621622  
2013  10.840455  28.729990  114.384615  
2014  10.648668  30.264610  134.638889  
2015  10.247821  29.158421  132.297297  
2016   8.548387  27.570367  134.627907  

[7 rows x 1304 columns]

## 02 Data Split

Train with 2010 - 2014, calibrate with 2015, test with 2016

In [14]:
siab_train = siab_s[siab_s.year < 2015] 
siab_calib = siab[siab.year == 2015] 
siab_test = siab[siab.year == 2016]

In [18]:
# Keep protected features, data from 2010-2014
#X_train_f = siab_train.iloc[:,4:164]

X_train = siab_train.iloc[:,4:164]

In [18]:
# Drop protected attributes, data from 2010-2014
#X_train_s = X_train.drop(
#    columns = ['frau1', 'maxdeutsch1', 'maxdeutsch.Missing.'])

In [19]:
# ltue, from siab_train, i.e. 2010-2014
y_train = siab_train.iloc[:, [3]]

In [20]:
# Keep protected features, data from 2015
X_calib = siab_calib.iloc[:,4:164]

# Drop protected features, data from 2015
#X_calib_s = X_calib_f.drop(
#    columns = ['frau1', 'maxdeutsch1', 'maxdeutsch.Missing.'])

# ltue, from siab_calib, i.e. 2015
y_calib = siab_calib.iloc[:, [3]]

In [21]:
# Keep protected features, data from 2016
X_test = siab_test.iloc[:,4:164]

# Drop protected features, data from 2016
#X_test_s = X_test_f.drop(
#    columns = ['frau1', 'maxdeutsch1', 'maxdeutsch.Missing.'])

# ltue, from siab_test, i.e. 2016
y_test = siab_test.iloc[:, [3]]

## 03 Descriptive Stats

In [22]:
siab_t = siab_train.copy(deep = True)
siab_t = pd.concat([siab_t, siab_calib, siab_test], ignore_index=True)

In [23]:
siab_t['nongerman'] = np.where(siab_t['maxdeutsch1'] == 0, 1, 0) # creates new column 'nongerman', if maxdeutsch1 == 0 then nongerman = 1, else 0 
siab_t.loc[siab_t['maxdeutsch.Missing.'] == 1, 'nongerman'] = np.nan # overwrite nongerman with NaN for any row where maxdeutsch.Missing. equals 1
siab_t['nongerman_male'] = np.where((siab_t['nongerman'] == 1) & (siab_t['frau1'] == 0), 1, 0)
siab_t['nongerman_female'] = np.where((siab_t['nongerman'] == 1) & (siab_t['frau1'] == 1), 1, 0)

In [24]:
desc1 = siab_t[['year', 'ltue']].groupby('year').mean()
desc1.to_latex('./output/desc1.tex', float_format = "%.3f") # Mean LTUE over time

In [25]:
desc2a = siab_t[['year', 'frau1', 'nongerman', 'nongerman_male', 'nongerman_female']].groupby(['year']).agg(['sum', 'count'])
desc2b = siab_t[['year', 'frau1', 'nongerman', 'nongerman_male', 'nongerman_female']].groupby(['year']).mean()
desc2c = siab_t[['year', 'ltue', 'frau1', 'nongerman', 'nongerman_male', 'nongerman_female']].groupby(['year', 'ltue']).agg(['mean', 'count'])

In [26]:
desc2a.to_latex('./output/desc2a.tex', float_format = "%.3f") # Number of cases over time
desc2b.to_latex('./output/desc2b.tex', float_format = "%.3f") # Socio-demo over time
desc2c.to_latex('./output/desc2c.tex', float_format = "%.3f") # Socio-demo by LTUE over time

## Save 

In [27]:
X_train.to_csv('./data/X_train.csv', index = False)
#X_train_s.to_csv('./output/X_train_s.csv', index = False)
y_train.to_csv('./data/y_train.csv', index = False)

X_calib.to_csv('./data/X_calib.csv', index = False)
#X_calib_s.to_csv('./output/X_calib_s.csv', index = False)
y_calib.to_csv('./data/y_calib.csv', index = False)

X_test.to_csv('./data/X_test.csv', index = False)
#X_test_s.to_csv('./output/X_test_s.csv', index = False)
y_test.to_csv('./data/y_test.csv', index = False)